# Feature extraction

Suppose we have the following directory structure. Data from this experiment was shown to the model during training. Images are saved as NPY files:

    /data/parsed/
        Experiment 001/
            Day 1/
                Sample A/
                    Replicate 1/
                        Class A/
                            A__32e88e1ac3a8f44bf8f77371155553b9.npy
                            A__3dc56a0c446942aa0da170acfa922091.npy
                        Class B/
                            B__8068ef7dcddd89da4ca9740bd2ccb31e.npy
        Experiment 002/
            Day 1/
                Sample A/
                    Replicate 1/
                        Class A/
                            A__8348deaa70dfc95c46bd02984d28b873.npy
                        Class B/
                            B__c1ecbca7bd98c01c1d3293b64cd6739a.npy
                            B__c56cfb8e7e7121dd822e47c67d07e2d4.npy
        ...

# User's settings

In [ ]:
input_dir = '/Data/STEP1_Parsing'
trained_model_location = '/Data/STEP2_Training'
output_dir = '/Data/STEP3_Extract'
feature_extraction_layer = 'pool5'
frame = 48 # frame size of each single-cell image

# Some hyperparameter
n_samples = None # sub-sampling for over-representing classes



# Some hyperparameter
n_samples = None # sub-sampling for over-representing classes

# Executable

In [ ]:
%matplotlib inline

import os, glob, re

import deepometry.model
import deepometry.utils
import deepometry.visualize
import matplotlib.pyplot
import pandas
import skimage.io

import keras.backend
import tensorflow

In [ ]:
def save_metadata(file, metadata_bkpr):
    with open(file, 'w') as f:
        f.write('Experiments\tDays\tSamples\tReplicates\tClasses\n')
        for i in range(len(metadata_bkpr)):
            f.write('{}\t{}\t{}\t{}\t{}\n'.format( metadata_bkpr[i][0], metadata_bkpr[i][1], metadata_bkpr[i][2], metadata_bkpr[i][3], metadata_bkpr[i][4]))

In [ ]:
configuration = tensorflow.ConfigProto()
configuration.gpu_options.allow_growth = True
# configuration.gpu_options.visible_device_list = "0"

session = tensorflow.Session(config=configuration)
keras.backend.set_session(session)

In [ ]:
pathnames_of_interest = [j for i in [x[0] for x in os.walk(input_dir)] for j in glob.glob(os.path.join(i,'*')) if '.npy' in j]

x, _, metadata_bkpr = deepometry.utils._load(pathnames_of_interest)

if os.path.isdir(trained_model_location):
    list_of_files = glob.glob(os.path.join(trained_model_location, '*_categories.h5')) # only file with correct naming, i.e.***_categories.h5, is accepted
    trained_model_location = max(list_of_files, key=os.path.getctime)

units = int( re.search('model_.*_([0-9]*)_categories.h5', os.path.basename(trained_model_location)).group(1) )

In [ ]:
model = deepometry.model.Model(shape=x.shape[1:], units=units)

model.compile()

# Extract the features
features = model.extract(x, feature_extraction_layer, trained_model_location, batch_size=32, standardize=True, verbose=1)

if len(features.shape) > 2:
    a = numpy.mean(features, axis = 1)
    del(features)
    features = numpy.mean(a, axis = 1)  

Once the data is loaded, we can redefine the model and extract the embedded features of `x`. The features will be saved as a TSV file using pandas. We omit the column headers and the data frame indexes from the exported data.

In [ ]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir5)   

# Export features to .TXT file, to be used as "metadata" on http://projector.tensorflow.org
# numpy.savetxt( os.path.join(output_dir, 'features_extracted_by_'+ feature_extraction_layer +'.txt'), features, delimiter='\t')

# Export features to .TSV file, to be used in local Tensorboard
features_df = pandas.DataFrame(data=features)
features_df.to_csv( os.path.join(output_dir, 'features_extracted_by_'+ feature_extraction_layer +'.tsv'), header=False, index=False, sep="\t")

print('Done')

Additional metadata can also be exported as a TSV. In this example, we export a single-column TSV containing the label data. Single-column metadata should exclude the column header (`header=False`). Metadata containing more than one column should include the column header (`header=True`).

In [ ]:
# Save labels, to be used as "metadata" on http://projector.tensorflow.org
save_metadata(os.path.join(output_dir, 'metadata.tsv'), metadata_bkpr)

# Visualize

Visualize an embedded feature spacing using TensorBoard or on the web-app http://projector.tensorflow.org. Use the outputs from `extract` to populate the features, metadata, and sprites parameters for `deepometry.visualize.make_projection`.

When using TensorBoard to visualize an embedding, you can display a "sprite" image for each data point. The sprite image is an NxN grid of image data. Use `deepometry.visualize.images_to_sprites` to create this grid. The `sprites_dim` parameter tells TensorBoard how big (in pixels) each sprite is. For example, if a single sprit is 48x48 pixels, then `sprites_dim=48`.

To launch TensorBoard, copy the output of the cell below and run it in your terminal.

In [ ]:
# Create the sprites image.
sprites = deepometry.visualize.images_to_sprite(x)

# Display the sprites image.
matplotlib.pyplot.rcParams["figure.figsize"] = (12, 12)
matplotlib.pyplot.imshow(sprites)

# Export the sprites image.
skimage.io.imsave(os.path.join(output_dir,'sprites.png'), sprites)

In [ ]:
features_file = os.path.abspath(os.path.join(output_dir,'*feature*'))
metadata_file = os.path.abspath(os.path.join(output_dir,'metadata.tsv'))
sprites_file = os.path.abspath(os.path.join(output_dir,'sprites.png'))
sprites_dim = frame

log_directory = deepometry.visualize.make_projection(
    features_file,
    metadata=metadata_file,
    sprites=sprites_file,
    sprites_dim=sprites_dim
)

print("tensorboard --logdir {:s}".format(log_directory))